In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
# nuclio: start-code

In [ ]:
%nuclio config kind = "job"
%nuclio config spec.build.baseImage = "python:3.6-jessie"

In [ ]:
%%nuclio cmd -c 
curl -sL https://aka.ms/InstallAzureCLIDeb > /tmp/getaz.sh
bash /tmp/getaz.sh
ls

In [ ]:
# nuclio: end-code

In [ ]:
# This function has no code

### mlconfig

In [ ]:
from mlrun import mlconf
import os

mlconf.dbpath = 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [ ]:
from mlrun import new_function,code_to_function, mount_v3io

# create job function object from notebook code
fn = new_function("azure_cli",mode="pass",kind='job')

## tests

In [ ]:
# Connection parameters
fn.set_env('AZURE_STORAGE_KEY','<AZURE_STORAGE_KEY>')
fn.set_env('AZURE_STORAGE_AUTH_MODE','<AZURE_STORAGE_KEY_AUTH_MODE>')
fn.set_env('AZURE_STORAGE_ACCOUNT','<AZURE_STORAGE_ACCOUNT>')
fn.set_env('AZURE_STORAGE_CONNECTION_STRING','<AZURE_STORAGE_CONNECTION_STRING>')

In [ ]:
# File upload parameters
azure_container = '<azurecontainer>'
local_file = "/User/data/wbanmasterlist.parquet"
target_blob = "wbanmasterlist.parquet"

In [ ]:
#Upload
#fn.spec.command = "az storage blob upload -f %s -c %s -n %s"% (local_file , azure_container, target_blob)

# Download single file
#fn.spec.command = "az storage blob download -f %s -c %s -n %s"% (local_file , azure_container, target_blob)

# Download batch
fn.spec.command = "az storage blob download-batch -d %s -s %s --pattern %s/*"% (local_file , azure_container, target_blob)
#az storage blob download-batch -d . -s mycontainer --pattern dir/*

In [ ]:
fn.apply(mount_v3io())

In [ ]:

#print(fn.to_yaml())

In [ ]:
### One time image build. Comment after image is created#
#fn.build_config(image=os.getenv('DOCKER_REGISTRY_SERVICE_HOST')+':5000/iguazio/azurecli',base_image='python:3.6-jessie',commands=['curl -sL https://aka.ms/InstallAzureCLIDeb > /tmp/getaz.sh && bash /tmp/getaz.sh'])
#fn.deploy()

In [ ]:
docker_registry = "<docker registry>"

In [ ]:
fn.spec.image=docker_registry + '/iguazio/azurecli'

In [ ]:
fn.run(project='azure-cli',watch=True)